# Books

1. [Extract CSVs from RSS](#extract-csvs-from-rss)
2. [Transform CSVs to DataFrames](#transform-csvs-to-dataframes)
3. [Load shelved\_to\_read\_books\_df](#load-shelved_to_read_books_df)
4. [(old) From HTML](#old-from-html)
5. [(discarded no avg ratings) From export csv](#discarded-no-avg-ratings-from-export-csv)
6. [(discarded) Logged in within Goodreads](#discarded-logged-in-within-goodreads)

## Extract CSVs from RSS

In [2]:
import pandas as pd
import urllib
from io import StringIO
from bs4 import BeautifulSoup
user_agent = "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"
shelf = "want-read-next" #%23ALL%23
url_prefix = "https://www.goodreads.com/review/list_rss/50196372?shelf="+shelf+"want-read-next&page="
all_books_df = pd.DataFrame()
previous_xml = ""
offset = 1 #
pages = 3
#shelves_to_query = [
#  {"shelf":"business","pages":"1"},
#  {"shelf":"finance-investing","pages":"2"},
#  {"shelf":"health","pages":"1"},
#  {"shelf":"history","pages":"1"},
#  {"shelf":"marketing","pages":"1"},
#  {"shelf":"philosophy","pages":"1"},
#  {"shelf":"psychology","pages":"2"},
#  {"shelf":"real-estate-inv","pages":"2"},
#  {"shelf":"specific-reference","pages":"1"}#
#]
for n in range(pages - offset): # starts in 0, but page in 1
  url = url_prefix + str(n+offset)
  req = urllib.request.Request(url)
  req.add_header('User-Agent', user_agent)
  xml = urllib.request.urlopen(req).read().decode('utf-8')
  if xml == previous_xml:
    print("XML is the same, breaking at", n)
    break
  previous_xml = xml
  try:
    books_df = pd.read_xml(StringIO(xml), xpath=".//item")
  except Exception as e:
    print("Error at", str(n+offset), ":", e[:e.find(".")], "...")
  fn = "../data/goodreads-javier-"+shelf+"-page-"+str(n+offset)+".csv" #shelves
  books_df.to_csv(fn)
  all_books_df = pd.concat([all_books_df, books_df])
all_books_df = all_books_df.drop_duplicates()
print("n_books",len(all_books_df)) # 681
print("cols",all_books_df.columns)

n_books 200
cols Index(['guid', 'pubDate', 'title', 'link', 'book_id', 'book_image_url',
       'book_small_image_url', 'book_medium_image_url', 'book_large_image_url',
       'book_description', 'book', 'author_name', 'isbn', 'user_name',
       'user_rating', 'user_read_at', 'user_date_added', 'user_date_created',
       'user_shelves', 'user_review', 'average_rating', 'book_published',
       'description'],
      dtype='object')


In [3]:
all_books_df

,guid,pubDate,title,link,book_id,book_image_url,book_small_image_url,book_medium_image_url,book_large_image_url,book_description,...,user_name,user_rating,user_read_at,user_date_added,user_date_created,user_shelves,user_review,average_rating,book_published,description
0,https://www.goodreads.com/review/show/58024143...,"Sun, 08 Sep 2024 23:19:40 -0700",The Millionaire Next Door: The Surprising Secr...,https://www.goodreads.com/review/show/58024143...,998,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,The incredible national bestseller that is cha...,...,Javier,0,None,"Sun, 08 Sep 2024 23:19:40 -0700","Mon, 28 Aug 2023 09:21:52 -0700","finance-investing, business, currently-reading",NaN,4.06,1995.0,"\n \n <a href=""https://www.goodreads..."
1,https://www.goodreads.com/review/show/47533785...,"Sun, 08 Sep 2024 23:19:28 -0700",Homo Deus: A History of Tomorrow,https://www.goodreads.com/review/show/47533785...,31138556,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,"<p>Yuval Noah Harari, author of the critically...",...,Javier,0,None,"Sun, 08 Sep 2024 23:19:28 -0700","Mon, 30 May 2022 08:29:23 -0700","history, currently-reading",NaN,4.20,2015.0,"\n \n <a href=""https://www.goodreads..."
2,https://www.goodreads.com/review/show/58024144...,"Sun, 08 Sep 2024 23:17:18 -0700",Real Estate Investing Gone Bad: 21 true storie...,https://www.goodreads.com/review/show/58024144...,28224284,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,Discover 21 true stories of real estate invest...,...,Javier,0,None,"Sun, 08 Sep 2024 23:17:18 -0700","Mon, 28 Aug 2023 09:21:56 -0700","real-estate-inv, want-read-next",NaN,4.00,NaN,"\n \n <a href=""https://www.goodreads..."
3,https://www.goodreads.com/review/show/58024029...,"Sun, 08 Sep 2024 23:16:45 -0700",Build A Rental Property Empire: The No-nonsens...,https://www.goodreads.com/review/show/58024029...,29739642,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,Everything you need to know about residential ...,...,Javier,0,None,"Sun, 08 Sep 2024 23:16:45 -0700","Mon, 28 Aug 2023 09:16:30 -0700","real-estate-inv, want-read-next",NaN,4.06,NaN,"\n \n <a href=""https://www.goodreads..."
4,https://www.goodreads.com/review/show/51376476...,"Sun, 08 Sep 2024 23:16:08 -0700",What Every Real Estate Investor Needs to Know ...,https://www.goodreads.com/review/show/51376476...,996,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,Real estate financial calculations made easy E...,...,Javier,0,None,"Sun, 08 Sep 2024 23:16:08 -0700","Thu, 01 Dec 2022 12:10:07 -0800","real-estate-inv, want-read-next",NaN,4.11,2003.0,"\n \n <a href=""https://www.goodreads..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://www.goodreads.com/review/show/68271163...,"Sat, 07 Sep 2024 09:37:24 -0700",Swipe to Unlock: The Primer on Technology and ...,https://www.goodreads.com/review/show/68271163...,43385933,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,<i>An alternative cover edition for this ISBN ...,...,Javier,0,NaN,"Sat, 07 Sep 2024 09:37:24 -0700","Sat, 07 Sep 2024 08:53:40 -0700","to-read, bus

In [ ]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd

user_agent = "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"

def get_n_ratings_from_bid(bid):
  url_prefix="https://www.goodreads.com/book/show/"
  url = url_prefix + str(bid)
  req = urllib.request.Request(url)
  req.add_header('User-Agent', user_agent)
  res = urllib.request.urlopen(req)
  html = res.read().decode('utf-8')
  soup =  BeautifulSoup(
    html,'html.parser', from_encoding="utf8"
  )
  n_ratings = list(
    soup.find(
      'span',
      attrs={'data-testid': 'ratingsCount'}
    ).children
  )[0]
  try:
    pages_format = list(
    soup.find(
        'p',
        attrs={'data-testid': 'pagesFormat'}
      ).children
    )[0]
  except Exception as e:
    print(str(e)[:128])
    pages_format=""
  if pages_format.find(" ")==-1:
    n_pages=0
  else:
    n_pages=pages_format[:pages_format.find(" ")]
  print(bid,",",n_ratings,",",n_pages)
  return n_ratings

print("bid , n_ratings , n_pages")

ids_to_check = pd.read_csv("../data/goodreads-transformed/ids_to_check.csv")
ids_to_check["book_id"].apply(lambda bid: get_n_ratings_from_bid(bid))
#rewrite as a loop with locs
#shelved_to_read_books_df = shelved_to_read_books_df[shelved_to_read_books_df["batch"]==0] #!temp
#shelved_to_read_books_df["n_ratings"] = shelved_to_read_books_df["book_id"].apply(
#  lambda bid: get_n_ratings_from_bid(bid)
#)

# shelved_to_read_books_df.to_csv("../data/goodreads-transformed/shelved_to_read_books_2_df.csv")

bid , n_ratings , n_pages


c:\Users\dark_\AppData\Local\Programs\Python\Python312\Lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


60505910 , 132 , 199
211876665 , 36 , 366
1168891 , 11 , 0
54204252 , 39 , 172
63327518 , 122 , 410
196239284 , 2,590 , 280
58612786 , 12,331 , 258
52326089 , 19 , 352
19168428 , 408 , 547
827031 , 2,217 , 131
41138379 , 665 , 288
626488 , 2 , 306
56991096 , 8 , 123
30966035 , 2 , 192
37811096 , 262 , 269
23791610 , 62 , 190
56931631 , 71 , 168
59575939 , 26,173 , 464
101255 , 15,771 , 896
58361319 , 501 , 344
44403965 , 1,450 , 534
56360682 , 273 , Kindle
1923820 , 296,343 , 1590
60292806 , 60 , 405
25663961 , 52,623 , 576
36067444 , 106 , 265
40630 , 6,525 , 312
55163317 , 32 , 771
79766 , 5,139 , 1171
30754045 , 3,651 , 384
482505 , 611 , 595
58109010 , 54 , 101
31170723 , 25,046 , 790
353435 , 1,181 , 128
25375430 , 10 , 202
45211636 , 2,075 , 434
37976541 , 265,541 , 339
55030 , 148,945 , 384
18693771 , 205,340 , 464
4069 , 748,363 , 165
63282345 , 100 , 288
452114 , 154 , 320
91349735 , 184 , 248
34466963 , 195,269 , 368
60190659 , 13,950 , 354
61153739 , 59,201 , 496
34890015 , 

## Transform CSVs to DataFrames

In [7]:
import pandas as pd
import urllib
from pathlib import Path

path = "../data/goodreads-want-read-next" # goodreads-to-read"
files = Path(path).glob('*.csv')
to_read_books_df = pd.concat(
  (pd.read_csv(f).assign(filename=f.stem) for f in files)
  , ignore_index=True
).drop_duplicates()

"""
cols = list(to_read_books_df.columns)
cols.sort()
print("columns")
print("\n".join(cols))
  columns
  author_name average_rating book book_description book_id book_image_url
  book_large_image_url book_medium_image_url book_published
  book_small_image_url description filename guid isbn link pubDate title
  user_date_added user_date_created user_name user_rating user_read_at
  user_review user_shelves
"""
del to_read_books_df["Unnamed: 0"]

to_read_books_df["date_shelved"] = to_read_books_df["user_date_added"].apply(
  lambda da: pd.to_datetime(da, format='mixed')
)
to_read_books_df = to_read_books_df[[
  "user_shelves", "book_published", "author_name", "title"
  , "average_rating", "user_rating", "book_id", "isbn"
  , "date_shelved", "filename"
]].sort_values("average_rating",ascending=False)\
.fillna(0).astype({"book_published":"int32"})
shelves_to_query = [
  "business", "finance-investing", "health", "history", "marketing"
  , "philosophy", "psychology", "real-estate-inv", "specific-reference"
]
shelved_to_read_books_df = pd.DataFrame()
for shelf in shelves_to_query:
  print(
    shelf, len(to_read_books_df[
      to_read_books_df["user_shelves"]\
        .str.contains(shelf)
    ])
  )
  shelved_to_read_books_df = pd.concat([
    shelved_to_read_books_df
    ,to_read_books_df[
      to_read_books_df["user_shelves"]\
        .str.contains(shelf)
    ]
  ])
shelved_to_read_books_df = shelved_to_read_books_df.groupby([
  "user_shelves", "book_published", "author_name", "title"
  , "average_rating", "user_rating", "book_id", "isbn", "date_shelved"
]).min().reset_index()#.sort_values("average_rating",ascending=False)
shelved_to_read_books_df.to_csv("../data/goodreads-transformed/shelved_to_read_books_df.csv")

business 33
finance-investing 27
health 14
history 6
marketing 9
philosophy 7
psychology 33
real-estate-inv 25
specific-reference 1


In [28]:
shelved_to_read_books_df

,Unnamed: 0,user_shelves,book_published,author_name,title,n_ratings_x,average_rating,darks,user_rating,book_id,isbn,date_shelved,filename,batch,#,spaces,n_ratings_y,n_pages
0,20,"history, currently-reading",2010,Laura Hillenbrand,"Unbroken: A World War II Story of Survival, Re...",956969,4.37,31.058697,0,8664353,1400064163,2024-08-16 12:05:04-07:00,goodreads-javier-to-read-page-4,3.0,1,,957016.0,0.0
1,55,"psychology, top-reference-books, want-read-next",1989,Stephen R. Covey,The 7 Habits of Highly Effective People: Power...,759365,4.16,29.066502,0,36072,0743269519,2024-09-07 07:24:06-07:00,goodreads-javier-to-read-page-4,3.0,2,,759443.0,0.0
2,26,"history, want-read-next",2003,Bill Bryson,A Short History of Nearly Everything,394475,4.21,28.916141,0,21,076790818X,2024-09-07 07:24:06-07:00,goodreads-javier-to-read-page-4,3.0,3,,394504.0,0.0
3,54,"psychology, top-reference-books, to-read",2012,Charles Duhigg,The Power of Habit: Why We Do What We Do in Li...,520355,4.13,28.489501,0,12609433,1400069289,2024-09-07 10:01:14-07:00,goodreads-javier-to-read-page-1,3.0,4,,520406.0,0.0
4,27,"history, want-read-next",2015,Yuval Noah Harari,Homo Deus: A History of Tomorrow,258765,4.20,28.465814,0,31138556,0,2024-09-07 07:24:07-07:00,goodreads-javier-to-read-page-4,3.0,5,,258806.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,129,"to-read, finance-investing",1991,Annette Thau,The Bond Book: Everything Investors Need to Kn...,0,3.96,15.681600,0,851083,0071358625,2024-09-07 09:42:48-07:00,goodreads-javier-to-read-page-2,2.0,302,,0.0,0.0
346,200,"to-read, marketing",0,Jim Vandehei,Smart Brevity: The Power of Saying More with Less,0,3.93,15.444900,0,59891634,1523516976,2024-09-07 10:17:15-07:00,goodreads-javier-to-read-page-1,2.0,303,,0.0,0.0
347,7,"finance-investing, missclick",0,José Luis Cava,El arte de especular,6,3.67,15.159096,0,33797837,8461117638,2024-09-07 09:46:18-07:00,goodreads-javier-to-read-page-1,0.0,0,0,0.0,0.0
348,58,"real-estate-inv, missclick",0,Arthur VanDam CPA,How to Finance Commercial Real Estate: How to ...,1,1.00,1.602060,0,29566707,189015833X,2024-09-07 09:46:19-07:00,goodreads-javier-to-read-page-1,0.0,0,0,0.0,0.0


In [105]:
import pandas as pd
import numpy as np

shelved_to_read_books_df = pd.read_csv(
  "../data/goodreads-transformed/shelved_to_read_books_df.csv"
)

try:
  del shelved_to_read_books_df["n_ratings"]
except:
  print("Source still didn't have ratings")

shelved_to_read_book_ids_df = pd.read_csv(
  "../data/goodreads-transformed/shelved_to_read_book_ids.csv"
)
shelved_to_read_books_df = shelved_to_read_books_df.merge(
  shelved_to_read_book_ids_df, how='left', on="book_id"
).fillna(0)

shelved_to_read_books_df.loc[shelved_to_read_books_df["book_id"]==9923513, "n_ratings"] = 4
shelved_to_read_books_df.loc[shelved_to_read_books_df["book_id"]==9923513, "average_rating"] = 4.3
shelved_to_read_books_df.loc[shelved_to_read_books_df["book_id"]==9923513, "n_pages"] = 196
shelved_to_read_books_df["n_pages"] = shelved_to_read_books_df["n_pages"].apply(lambda np: 200 if np==0 else np)
shelved_to_read_books_df["n_ratings"] = shelved_to_read_books_df["n_ratings"].apply(lambda nr: 300 if nr==0 else nr)

shelved_to_read_books_df = shelved_to_read_books_df\
  .astype({"n_ratings": "int32", "n_pages": "int32"})

def calc_darks(r):
  """avg>pages>reads"""
  avg_weight = max(max(r.iloc[0], r.iloc[0]**2-12)-3,-1)
  n_ratings_weight = max(1.5*np.log10(r.iloc[1]+1)-0.18,0)
  n_pages_weight = max(21.4-8*np.log10(r.iloc[2]+1),0)
  return avg_weight + n_ratings_weight + n_pages_weight
shelved_to_read_books_df["darks"] = shelved_to_read_books_df[[
  "average_rating", "n_ratings", "n_pages"
]].apply(lambda r: calc_darks(r), axis=1)

shelved_to_read_books_df = shelved_to_read_books_df.sort_values(
  "darks", ascending=[False]
)[[
  "user_shelves", "book_published", "author_name", "title"
  , "n_ratings", "average_rating", "n_pages", "darks", "user_rating"
  , "book_id", "isbn", "date_shelved", "filename"
]][
  ~shelved_to_read_books_df["user_shelves"].str.contains("read-inspiring") &
  (shelved_to_read_books_df["user_rating"]!=1) &
  (shelved_to_read_books_df["user_rating"]!=2)
]
shelved_to_read_books_df.index.name='index'
shelved_to_read_books_df = shelved_to_read_books_df.reset_index()
shelved_to_read_books_df.to_csv(
  '../data/goodreads-transformed/shelved_to_read_books_2_df.csv'
)
shelved_to_read_books_df

Source still didn't have ratings


index                                   user_shelves  book_published  \
0        0                     history, currently-reading            2010   
1       13                       missclick, mental-health            2016   
2        7                            psychology, to-read            1959   
3       11                          philosophy, missclick            -400   
4        5  psychology, finance-investing, want-read-next            2016   
..     ...                                            ...             ...   
345    346                             to-read, marketing               0   
346     80                             specific-reference            2008   
347    347                   finance-investing, missclick               0   
348    348                     real-estate-inv, missclick               0   
349    349                     real-estate-inv, missclick               0   

           author_name                                              title  \
0    Laura Hillenbrand  Unbroken: A World War II Story of Survival, Re...   
1          Mark Manson  The Subtle Art of Not Giving a F*ck: A Counter...   
2    David J. Schwartz                          The Magic of Thinking Big   
3              Sun Tzu                                     The Art of War   
4         J.L. Collins  The Simple Path to Wealth: Your road map to fi...   
..                 ...                                                ...   
345       Jim Vandehei  Smart Brevity: The Power of Saying More with Less   
346        Donald Voet                         Principles of Biochemistry   
347     José Luis Cava                               El arte de especular   
348  Arthur VanDam CPA  How to Finance Commercial Real Estate: How to ...   
349    Lawrence Uchill  Anatomy of a Mortgage: Understanding and Negot...   

     n_ratings  average_rating  n_pages      darks  user_rating   book_id  \
0       957024            4.37      475  16.815034            0   8664353   
1      1184569            3.88      200  15.941194            0  28257707   
2        94202            4.25      238  15.262567            0    759945   
3       507300            3.98      200  15.200017            0     10534   
4        25033            4.43      288  15.016067            0  30646587   
..         ...             ...      ...        ...          ...       ...   
345          0            3.93      200  -2.098328            0  59891634   
346         70            4.09     1209  -3.102773            4   5489002   
347          6            3.67      479  -4.270194            0  33797837   
348          1            1.00      111  -6.589090            0  29566707   
349          0            0.00      200 -11.031961            0  35873297   

           isbn               date_shelved                         filename  \
0    1400064163  2024-08-16 12:05:04-07:00  goodreads-javier-to-read-page-4   
1             0  2024-09-07 09:54:15-07:00  goodreads-javier-to-read-page-1   
2    0671646788  2023-11-19 10:00:04-08:00  goodreads-javier-to-read-page-4   
3             0  2024-09-07 09:18:46-07:00  goodreads-javier-to-read-page-3   
4             0  2024-09-07 07:24:09-07:00  goodreads-javier-to-read-page-4   
..          ...                        ...                              ...   
345  1523516976  2024-09-07 10:17:15-07:00  goodreads-javier-to-read-page-1   
346  0470233966  2023-04-23 02:16:06-07:00  goodreads-javier-to-read-page-5   
347  8461117638  2024-09-07 09:46:18-07:00  goodreads-javier-to-read-page-1   
348  189015833X  2024-09-07 09:46:19-07:00  goodreads-javier-to-read-page-1   
349  1634257693  2024-09-07 09:46:18-07:00  goodreads-javier-to-read-page-1   

    batch  
0          
1          
2          
3          
4          
..    ...  
345        
346        
347        
348        
349        

[350 rows x 15 columns]

In [113]:
ids_to_check = shelved_to_read_books_df[
  (shelved_to_read_books_df["average_rating"]>=4) &
  (shelved_to_read_books_df["n_ratings"]==300) &
  (shelved_to_read_books_df["n_pages"]==200)
]["book_id"]
ids_to_check.to_csv("ids_to_check.csv")

## Load shelved_to_read_books_df

In [4]:
try:
  del shelved_to_read_books_df["Unnamed: 0"]
except:
  print("Index already cleared for shelved_to_read_books_df")
shelved_to_read_books_df.head()#[:0].T

,Unnamed: 0,index,user_shelves,book_published,author_name,title,n_ratings,average_rating,n_pages,darks,user_rating,book_id,isbn,date_shelved,filename
0,0,138,"to-read, philosophy",1946,Viktor E. Frankl,Man’s Search for Meaning,748363,4.38,165,21.198891,0,4069,080701429X,2024-09-07 09:55:00-07:00,goodreads-javier-want-read-next-page-1
1,1,172,"top-reference-books, psychology, want-read-next",2008,Randy Pausch,The Last Lecture,350427,4.25,206,17.598444,0,2318271,1401323251,2024-09-07 14:12:30-07:00,goodreads-javier-want-read-next-page-1
2,2,4,"business, marketing, want-read-next",1968,Og Mandino,The Greatest Salesman in the World,65073,4.21,118,17.312407,0,356896,055327757X,2024-09-08 22:49:56-07:00,goodreads-javier-want-read-next-page-1
3,3,2,"business, history, want-read-next",2018,John Carreyrou,Bad Blood: Secrets and Lies in a Silicon Valle...,265541,4.39,339,16.864898,0,37976541,0,2024-09-08 22:43:09-07:00,goodreads-javier-want-read-next-page-1
4,4,43,"marketing, currently-reading, business",0,Alex Hormozi,$100M Offers: How To Make Offers So Good Peopl...,12331,4.59,258,16.715213,0,58612786,1737475707,2024-09-08 09:51:04-07:00,goodreads-javier-want-read-next-page-1


In [10]:
import pandas as pd
shelved_to_read_books_df = pd.read_csv('../data/goodreads-transformed/shelved_to_read_books_2_df.csv')
shelved_to_read_books_df["count"] = 1
author_counts_df = shelved_to_read_books_df[["author_name","count"]]\
  .groupby("author_name")\
  .sum().reset_index().sort_values("count",ascending=False)
author_scores_df = shelved_to_read_books_df[["author_name","darks"]]\
  .rename(columns={"darks": "author_darks"}).groupby("author_name")\
  .mean().reset_index().sort_values("author_darks",ascending=False)
author_top_book_df = shelved_to_read_books_df[[
  "author_name", "user_shelves", "title", "average_rating"
  , "n_pages", "n_ratings", "darks", "user_rating"
]].groupby("author_name").first()
top_authors_df = author_counts_df.merge(
  author_scores_df, how='left', on="author_name"
)
top_authors_df = top_authors_df.merge(
  author_top_book_df, how='left', on="author_name"
).sort_values("author_darks",ascending=False)
top_authors_df.to_csv("../data/goodreads-transformed/top_authors.csv")
top_authors_df

,author_name,count,author_darks,user_shelves,title,average_rating,n_pages,n_ratings,darks,user_rating
77,Mark Manson,1,15.941194,"missclick, mental-health",The Subtle Art of Not Giving a F*ck: A Counter...,3.88,200,1184569,15.941194,0
104,David J. Schwartz,1,15.387622,"psychology, want-read-next",The Magic of Thinking Big,4.26,238,94202,15.387622,0
106,Eric Jorgenson,1,14.843123,"finance-investing, to-read",El Almanaque de Naval Ravikant,4.22,200,51041,14.843123,0
59,Spencer Johnson,1,14.733351,"business, psychology, want-read-next",Who Moved My Cheese?,3.87,200,481911,14.733351,0
103,Alexander Osterwalder,1,14.731793,"business, want-read-next",Business Model Generation,4.20,200,56640,14.731793,0
...,...,...,...,...,...,...,...,...,...,...
98,Annette Thau,1,-1.988965,"to-read, finance-investing",The Bond Book: Everything Investors Need to Kn...,3.96,200,0,-1.988965,0
120,Jim Vandehei,1,-2.098328,"to-read, marketing",Smart Brevity: The Power of Saying More with Less,3.93,200,0,-2.098328,0
114,José Luis Cava,1,-4.270194,"finance-investing, missclick",El arte de especular,3.67,479,6,-4.270194,0
97,Arthur VanDam CPA,1,-6.589090,"real-estate-inv, missclick",How to Finance Commercial Real Estate: How to ...,1.00,111,1,-6.589090,0


In [8]:
shelved_to_read_books_df[shelved_to_read_books_df["author_name"]=="J.L. Collins"]

,Unnamed: 0,index,user_shelves,book_published,author_name,title,n_ratings,average_rating,n_pages,darks,user_rating,book_id,isbn,date_shelved,filename
31,31,5,"psychology, finance-investing, want-read-next",2016,J.L. Collins,The Simple Path to Wealth: Your road map to fi...,25033,4.43,288,12.755513,0,30646587,0,2024-09-07 07:24:09-07:00,goodreads-javier-to-read-page-4


In [6]:
shelved_to_read_books_df[
  (shelved_to_read_books_df["n_ratings"]==0) |
  (shelved_to_read_books_df["average_rating"]==0)
]["book_id"].to_csv("data/goodreads-transformed/goodreads_empty_ids.csv")

## (old) From HTML

In [ ]:
import numpy as np
import os
import pandas as pd
import sys
import urllib
from bs4 import BeautifulSoup
from datetime import datetime
from io import StringIO

path = '../data/goodreads_transformed/books.html'
soup = BeautifulSoup(open(path),'html.parser', from_encoding="utf8")

# list_header = []
# header = soup.find_all("table")[0].find("tr")
# for items in header:
#     try:
#         list_header.append(items.get_text())
#     except:
#         continue

data = []
html_data = soup.find_all("table")[0]
tr_data=html_data.find_all("tr")[1:]

books_html_df = pd.read_xml(StringIO(str("".join(tr_data))))
books_html_df

#for element in html_data:
#    sub_data = []
#    for sub_element in element:
#        try:
#            sub_data.append(sub_element.get_text())
#        except:
#            continue
#    data.append(sub_data)
#raw_books_df = pd.DataFrame(data = data, columns = list_header).dropna()
#raw_books_df

In [ ]:
books_df = raw_books_df[[
  'title\n \n\n'
  ,'author\n \n*\n\n'
  #,'isbn\n\n\n'
  #,'isbn13\n\n\n'
  #,'asin\n\n\n'
  ,'avg rating\n\n\n'
  ,'num ratings\n\n\n'
  ,'num pages\n\n\npp\n\n\n'
  ,'date pub\n\n\n'
  #,'date pub edition\n\n\n'
  #,'my rating\n\n\n\n\n\n'
  ,'shelves\n\n\n\n'
  #,'review\n\n\n\n\n'
  #,'notes\n\n\n\n\n'
  #,'comments\n\n\n\n'
  #,'votes\n\n\n\n'
  #,'# times read\n\n\n'
  ,'date started\n\n\n'
  ,'date read\n\n\n'
  ,'date added\n\n\n\n\n'
  #,'owned\n\n'
  #,'format\n\n        Kindle Edition\n        [edit]\n\n'
  #,'actions\n\n\n\nedit\n\nview\n               \n\n\nremove book\n\n\n\n'
]].rename(columns={
    "title\n \n\n":"title"
    ,"author\n \n*\n\n":"author"
    #,"isbn\n\n\n":"isbn"
    #,"isbn13\n\n\n":"isbn13"
    #,"asin\n\n\n":"asin"
    ,"avg rating\n\n\n":"avg rating"
    ,"num ratings\n\n\n":"num ratings"
    ,"num pages\n\n\npp\n\n\n":"num pages"
    ,"date pub\n\n\n":"date pub"
    #,"date pub edition\n\n\n":"date pub edition"
    #,"my rating\n\n\n\n\n\n":"my rating"
    ,"shelves\n\n\n\n":"shelves"
    #,"review\n\n\n\n\n":"review"
    #,"notes\n\n\n\n\n":"notes"
    #,"comments\n\n\n\n":"comments"
    #,"votes\n\n\n\n":"votes"
    #,"# times read\n\n\n":"# times read"
    ,"date started\n\n\n":"date started"
    ,"date read\n\n\n":"date read"
    ,"date added\n\n\n\n\n":"date added"
    #,"owned\n\n":"owned"
    #,"format\n\n        Kindle Edition\n        [edit]\n\n":"format"
    #,"actions\n\n\n\nedit\n\nview\n               \n\n\nremove book\n\n\n\n":"actions"
})
#for col in books_df.columns:
#  if is_string_dtype(f[col]):
#    f[col] = f[col].str.replace(
#                            "[^-$A-Za-z0-9 <>=\":#,?%!*'\(\);&\+áéíóúñèößºª\[\/\]\.]+", "")
books_df = books_df.replace({r"[^-$A-Za-z0-9 <>=\":#,?%!*'\(\);&\+áéíóúñèößºª]+": ""}, regex=True)
books_df = books_df.replace({
  "title ": "", "author ": "", "avg rating ": "", "num ratings ": "", "num pages ": "", " pp": ""
  ,"date pub ": "", "shelves": "", "\[edit\]": "", "date started": "", "date read": ""
  ,"date added ": "", "                ": " ", "        ": " ", "    ": " "
  ,"   ": " ", "  ": " "
}, regex=True)
books_df["num ratings"] = books_df["num ratings"].replace({",": ""}, regex=True)
books_df["num pages"] = books_df["num pages"].replace({",": "", "unknown": "200"}, regex=True)
books_df["date pub"] = books_df["date pub"].replace({"not set": "", "unknown": ""}, regex=True)
books_df["date started"] = books_df["date started"].replace({"not set": ""}, regex=True)
books_df["date read"] = books_df["date read"].replace({"not set": ""}, regex=True)
books_df["date added"] = books_df["date added"].replace({"not set": ""}, regex=True)
books_df = books_df.astype({
  "avg rating": "float32",
  "num ratings": "int32",
  "num pages": "int32"
})
date_pub_fixed = pd.to_datetime(books_df["date pub"],format='mixed',errors='coerce') #format='%b %d, %Y'
date_pub_fixed = date_pub_fixed.fillna(pd.to_datetime(books_df["date pub"],format='%Y',errors='coerce'))
books_df['date_pub'] = date_pub_fixed.fillna(datetime(1900, 1, 1, 0))
del books_df["date pub"]
books_df["date_started"] = pd.to_datetime(books_df["date started"],format='mixed',errors='coerce').fillna(datetime(1900, 1, 1, 0))
del books_df["date started"]
books_df["date_read"] = pd.to_datetime(books_df["date read"],format='mixed',errors='coerce').fillna(datetime(1900, 1, 1, 0))
del books_df["date read"]
books_df["date_added"] = pd.to_datetime(books_df["date added"],format='mixed',errors='coerce').fillna(datetime(1900, 1, 1, 0))
del books_df["date added"]
books_df["darks"] = books_df[[
  "avg rating", "num ratings", "num pages"
]].apply(
  lambda r: max(r.iloc[0]**2 + 2*np.log10(r.iloc[1]+1) - 2*np.log10(r.iloc[2]+1),0) # avg>pages>reads
  # max(np.log10((r.iloc[0]*r.iloc[1])+1) - np.log10(r.iloc[2]+1),0) strategy weighting more pages>reads>avg
  , axis=1
)
books_df = books_df.sort_values("darks", ascending=[False])
books_df.to_csv('../data/goodreads_transformed/books_df.csv')
filtered_books_df = books_df[
  ~books_df["shelves"].str.contains("novels") &
  ~books_df["shelves"].str.contains("missclick")
].reset_index()
filtered_books_df.index.name='index'
filtered_books_df.to_excel("../data/goodreads_transformed/filtered_books_1_df.xlsx")
filtered_books_df.dtypes

In [ ]:
filtered_books_df[
  books_df["shelves"].str.contains("read-next-shortlist") | #user_shelves
  books_df["shelves"].str.contains("currently-reading")
]
filtered_books_df

C:\Users\dark_\AppData\Local\Temp\ipykernel_5928\2200323226.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_books_df[


,title,author,avg rating,num ratings,num pages,shelves,date started,date read,date added,date_pub,darks
39,"Unbroken: A World War II Story of Survival, R...","Hillenbrand, Laura",4.38,954708,475,"read-next-shortlist, history","Aug 16, 2024",not set,"Apr 19, 2022",2010-11-16,25.788929
33,The Last Lecture,"Pausch, Randy",4.26,349713,206,"read-next-shortlist, top-reference-books","Aug 16, 2024",not set,"Sep 04, 2017",2008-04-08,24.603087
70,The Psychology of Money,"Housel, Morgan",4.30,205232,252,"read-next-shortlist, psychology","Aug 16, 2024",not set,"May 30, 2022",2020-01-01,24.308255
22,"Foundation (Foundation, #1)","Asimov, Isaac",4.17,550325,244,"read-next-shortlist, novels","Aug 16, 2024",not set,"Nov 29, 2020",1951-08-30,24.091809
38,The Big Short: Inside the Doomsday Machine,"Lewis, Michael",4.30,162351,264,"read-next-shortlist, financial-investing","Aug 16, 2024",not set,"Sep 15, 2022",2010-03-15,24.064425
6,The 7 Habits of Highly Effective People: Powe...,"Covey, Stephen R.",4.16,756470,372,"read-next-shortlist, psychology, top-reference...","Aug 10, 2024",not set,"Sep 04, 2017",1994-03-01,23.919766
23,A Short History of Nearly Everything,"Bryson, Bill",4.22,393186,544,"read-next-shortlist, history","Aug 16, 2024",not set,"Sep 12, 2020",2003-01-01,23.524804
60,The Simple Path to Wealth: Your road map to f...,"Collins, J L",4.43,24775,288,"read-next-shortlist, financial-investing, psyc...","Nov 19, 2023",not set,"Nov 19, 2023",2016-06-17,23.491165
54,Homo Deus: A History of Tomorrow,"Harari, Yuval Noah",4.20,257313,450,"read-next-shortlist, history","Aug 10, 2024",not set,"May 30, 2022",2015-01-01,23.152572
40,The Lean Startup (Hardcover),"Ries, Eric*",4.11,335245,299,"read-next-shortlist, top-reference-books","Aug 10, 2024",not set,"May 30, 2022",2011-01-01,22.988586


## (discarded no avg ratings) From export csv

In [ ]:
import pandas as pd

books_export_df = pd.read_csv("../data/goodreads_library_export.csv")
print("n_books",len(books_export_df))
print("columns",books_export_df.columns)
books_export_excerpt_df = books_export_df[[
  "Author l-f", "Title", "Read Count", "Average Rating", "Number of Pages"
  ,"My Rating", "Bookshelves", "Exclusive Shelf"
  ,"Original Publication Year", "Book Id"
]].fillna(0).astype({
  "Number of Pages": "int32",
  "Original Publication Year": "int32"
})
books_export_excerpt_df.head()

n_books 509
columns Index(['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors',
       'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
       'Number of Pages', 'Year Published', 'Original Publication Year',
       'Date Read', 'Date Added', 'Bookshelves', 'Bookshelves with positions',
       'Exclusive Shelf', 'My Review', 'Spoiler', 'Private Notes',
       'Read Count', 'Owned Copies'],
      dtype='object')


,Author l-f,Title,Read Count,Average Rating,Number of Pages,My Rating,Bookshelves,Exclusive Shelf,Original Publication Year,Book Id
0,"Cixin, Liu","Death's End (Remembrance of Earth’s Past, #3)",0,4.42,604,0,to-read,to-read,2010,25451264
1,"Cixin, Liu","The Dark Forest (Remembrance of Earth’s Past, #2)",0,4.41,512,0,to-read,to-read,2008,23168817
2,"Cixin, Liu",The Three-Body Problem (Remembrance of Earth’s...,0,4.08,472,0,"novels, to-read",to-read,2006,20518872
3,"Tolle, Eckhart",The Power of Now: A Guide to Spiritual Enlight...,0,4.15,229,0,"psychology, to-read",to-read,1997,6708
4,"Thiel, Peter","Zero to One: Notes on Startups, or How to Buil...",0,4.15,195,0,"top-reference-books, to-read",to-read,2014,18050143


In [87]:
cols = list(books_export_excerpt_df.columns)
cols.sort()
print("columns")
print("\n".join(cols))

columns
Additional Authors
Author
Author l-f
Average Rating
Binding
Book Id
Bookshelves
Bookshelves with positions
Date Added
Date Read
Exclusive Shelf
ISBN
ISBN13
My Rating
My Review
Number of Pages
Original Publication Year
Owned Copies
Private Notes
Publisher
Read Count
Spoiler
Title
Year Published


In [88]:
books_df = books_export_excerpt_df[books_export_excerpt_df["Exclusive Shelf"]!="missclick"][[
  "Author l-f", "Title", "Read Count", "Average Rating", "Number of Pages"
  ,"My Rating", "Bookshelves", "Exclusive Shelf"
  ,"Original Publication Year", "Book Id", "ISBN"
]].fillna(0).astype({
  "Number of Pages": "int32",
  "Original Publication Year": "int32"
}).sort_values(
  ["Original Publication Year", "Author l-f", "Title"]
)
books_df

,Author l-f,Title,Read Count,Average Rating,Number of Pages,My Rating,Bookshelves,Exclusive Shelf,Original Publication Year,Book Id,ISBN
336,"Abbott, Devin",Fullstack React Native: Create beautiful mobil...,0,4.15,688,0,"to-read, informatics",to-read,0,43628062,"=""1728995558"""
348,"Accomazzo, Anthony",Fullstack React: The Complete Guide to ReactJS...,0,4.24,830,0,"to-read, informatics",to-read,0,32705383,"=""0991344626"""
270,"Antani, Ved",JavaScript: Functional Programming for JavaScr...,0,4.25,635,0,"to-read, javascript",to-read,0,31863150,"=""1787125572"""
238,"Arnell, R. Scott",Sustainable & Responsible Investing 360°: Less...,0,4.28,404,0,"financial-investing, to-read",to-read,0,62475608,"=""1538149044"""
113,"Arranz, Ana Monmeneu",VENDE TU CASA Y COMPRA TU LIBERTAD: Guía rápid...,0,3.71,182,0,to-read,to-read,0,71943901,"="""""
...,...,...,...,...,...,...,...,...,...,...,...
120,"Schur, Michael",How to Be Perfect: The Correct Answer to Every...,0,4.14,304,0,"to-read, psychology",to-read,2022,58484901,"=""1982159316"""
119,"Yong, Ed",An Immense World: How Animal Senses Reveal the...,0,4.47,464,0,"to-read, specific-reference",to-read,2022,59575939,"="""""
95,"Attia, Peter",Outlive: The Science & Art of Longevity,0,4.37,496,0,to-read,to-read,2023,61153739,"=""0593236599"""
96,"Desmond, Matthew","Poverty, by America",0,4.27,284,0,to-read,to-read,2023,61358638,"=""0593239911"""


In [68]:
rss_cols = list(all_books_df.columns)
rss_cols.sort()
print("\n".join(rss_cols))

author_name
average_rating
book
book_description
book_id
book_image_url
book_large_image_url
book_medium_image_url
book_published
book_small_image_url
description
guid
isbn
link
pubDate
title
user_date_added
user_date_created
user_name
user_rating
user_read_at
user_review
user_shelves


In [80]:
all_books_df["date_shelved"] = all_books_df["user_date_added"].apply(
  lambda da: pd.to_datetime(
    da, format='mixed' #'%a, %d %b %Y %H:%M:%S %Z'
  )#.strftime('%Y%M%D%H%M')
)
all_books_df = all_books_df.fillna(0).astype({
  "book_published": "int32"
})

latest_books_df = all_books_df[[
  "book_published", "user_shelves", "author_name", "title", "book_id",
  "date_shelved", "average_rating", "user_rating",  #!, "ratings_number" pending
]].sort_values("date_shelved",ascending=False)
latest_books_df

,book_published,user_shelves,author_name,title,book_id,date_shelved,average_rating,user_rating
0,1999,to-read,William J. Poorvu,The Real Estate Game: The Intelligent Guide To...,643390,2024-09-07 05:52:38-07:00,4.13,0
1,2013,to-read,Stephen Wendel,Designing for Behavior Change: Applying Psycho...,18167241,2024-09-07 01:19:35-07:00,4.08,0
2,0,to-read,Jakub Lasak,"Theory of People: Master Behaviors, Business, ...",50893389,2024-09-07 01:19:23-07:00,4.23,0
3,2016,to-read,Richard H. Thaler,Misbehaving: The Making of Behavioral Economics,26530355,2024-09-07 01:18:54-07:00,4.16,0
4,0,to-read,Walter Grant,How to Start a Successful Airbnb Business: Qui...,59335141,2024-09-03 21:13:39-07:00,4.30,0
...,...,...,...,...,...,...,...,...
74,2019,missclick,Clea Shearer,The Home Edit: A Guide to Organizing and Reali...,40712502,2021-01-16 05:23:12-08:00,3.77,0
75,2010,missclick,Marie Kondō,The Life-Changing Magic of Tidying Up: The Jap...,22318578,2021-01-16 04:39:28-08:00,3.88,0
76,1651,missclick,Thomas Hobbes,Leviathan,91953,2021-01-16 04:39:15-08:00,3.71,0
77,2019,missclick,Sally Thorne,99 Percent Mine,36300625,2020-09-12 05:14:54-07:00,3.32,0


In [31]:
books_df["Book Id"]

336     43628062
348     32705383
270     31863150
238     62475608
113     71943901
         ...    
120     58484901
119     59575939
95      61153739
96      61358638
104    201545491
Name: Book Id, Length: 412, dtype: int64

In [33]:
len(books_df[books_df["Exclusive Shelf"]=="to-read"])

345

In [ ]:
url = "https://www.goodreads.com/tooltips?"
for bn, bid in enumerate(book_ids):
  if not (bn+1)%20:
    url += "use_wtr_tooltip=true\n\nhttps://www.goodreads.com/tooltips?"
  url += "resources%5BBook."+str(bid)+"%5D%5Btype%5D=Book&"
url_f = open("url.txt", "w+", encoding="utf-8")
url_f.write(url)
url_f.close()

In [ ]:
books_all_cols_no_missclick_by_pub_year_df = books_export_excerpt_df[books_export_excerpt_df["Exclusive Shelf"]!="missclick"][[
  "Author l-f", "Title", "Read Count", "Average Rating", "Number of Pages"
  ,"My Rating", "Bookshelves", "Exclusive Shelf"
  ,"Original Publication Year", "Book Id", "ISBN"
]].fillna(0).astype({
  "Number of Pages": "int32",
  "Original Publication Year": "int32"
}).sort_values(
  ["Original Publication Year", "Author l-f", "Title"]
)
books_all_cols_no_missclick_by_pub_year_df

,Author l-f,Title,Read Count,Average Rating,Number of Pages,My Rating,Bookshelves,Exclusive Shelf,Original Publication Year,Book Id,ISBN
336,"Abbott, Devin",Fullstack React Native: Create beautiful mobil...,0,4.15,688,0,"to-read, informatics",to-read,0,43628062,"=""1728995558"""
348,"Accomazzo, Anthony",Fullstack React: The Complete Guide to ReactJS...,0,4.24,830,0,"to-read, informatics",to-read,0,32705383,"=""0991344626"""
270,"Antani, Ved",JavaScript: Functional Programming for JavaScr...,0,4.25,635,0,"to-read, javascript",to-read,0,31863150,"=""1787125572"""
238,"Arnell, R. Scott",Sustainable & Responsible Investing 360°: Less...,0,4.28,404,0,"financial-investing, to-read",to-read,0,62475608,"=""1538149044"""
113,"Arranz, Ana Monmeneu",VENDE TU CASA Y COMPRA TU LIBERTAD: Guía rápid...,0,3.71,182,0,to-read,to-read,0,71943901,"="""""
...,...,...,...,...,...,...,...,...,...,...,...
120,"Schur, Michael",How to Be Perfect: The Correct Answer to Every...,0,4.14,304,0,"to-read, psychology",to-read,2022,58484901,"=""1982159316"""
119,"Yong, Ed",An Immense World: How Animal Senses Reveal the...,0,4.47,464,0,"to-read, specific-reference",to-read,2022,59575939,"="""""
95,"Attia, Peter",Outlive: The Science & Art of Longevity,0,4.37,496,0,to-read,to-read,2023,61153739,"=""0593236599"""
96,"Desmond, Matthew","Poverty, by America",0,4.27,284,0,to-read,to-read,2023,61358638,"=""0593239911"""


In [ ]:
cols = list(books_export_excerpt_df.columns)
cols.sort()
print("columns\n","\n".join(cols))
books_df = books_export_excerpt_df[books_export_excerpt_df["Exclusive Shelf"]!="missclick"][[
  "Author l-f", "Title", "Read Count", "Average Rating", "Number of Pages"
  ,"My Rating", "Bookshelves", "Exclusive Shelf"
  ,"Original Publication Year", "Book Id", "ISBN"
]].fillna(0).astype({
  "Number of Pages": "int32",
  "Original Publication Year": "int32"
}).sort_values(
  ["Original Publication Year", "Author l-f", "Title"]
)
books_df

columns
 Additional Authors
Author
Author l-f
Average Rating
Binding
Book Id
Bookshelves
Bookshelves with positions
Date Added
Date Read
Exclusive Shelf
ISBN
ISBN13
My Rating
My Review
Number of Pages
Original Publication Year
Owned Copies
Private Notes
Publisher
Read Count
Spoiler
Title
Year Published


,Author l-f,Title,Read Count,Average Rating,Number of Pages,My Rating,Bookshelves,Exclusive Shelf,Original Publication Year,Book Id,ISBN
336,"Abbott, Devin",Fullstack React Native: Create beautiful mobil...,0,4.15,688,0,"to-read, informatics",to-read,0,43628062,"=""1728995558"""
348,"Accomazzo, Anthony",Fullstack React: The Complete Guide to ReactJS...,0,4.24,830,0,"to-read, informatics",to-read,0,32705383,"=""0991344626"""
270,"Antani, Ved",JavaScript: Functional Programming for JavaScr...,0,4.25,635,0,"to-read, javascript",to-read,0,31863150,"=""1787125572"""
238,"Arnell, R. Scott",Sustainable & Responsible Investing 360°: Less...,0,4.28,404,0,"financial-investing, to-read",to-read,0,62475608,"=""1538149044"""
113,"Arranz, Ana Monmeneu",VENDE TU CASA Y COMPRA TU LIBERTAD: Guía rápid...,0,3.71,182,0,to-read,to-read,0,71943901,"="""""
...,...,...,...,...,...,...,...,...,...,...,...
120,"Schur, Michael",How to Be Perfect: The Correct Answer to Every...,0,4.14,304,0,"to-read, psychology",to-read,2022,58484901,"=""1982159316"""
119,"Yong, Ed",An Immense World: How Animal Senses Reveal the...,0,4.47,464,0,"to-read, specific-reference",to-read,2022,59575939,"="""""
95,"Attia, Peter",Outlive: The Science & Art of Longevity,0,4.37,496,0,to-read,to-read,2023,61153739,"=""0593236599"""
96,"Desmond, Matthew","Poverty, by America",0,4.27,284,0,to-read,to-read,2023,61358638,"=""0593239911"""


## (discarded) Logged in within Goodreads

In [23]:
import urllib
from bs4 import BeautifulSoup

url="https://www.goodreads.com/review/list/50196372-javier"
req = urllib.request.Request(url)
req.add_header('User-Agent', 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7')
response = urllib.request.urlopen(req)
data = response.read() # a `bytes` object
html = data.decode('utf-8') # a `str`; this step can't be used if data is binary
f = open("goodreads-javier.html", "w+", encoding="utf-8")
f.write(html)
f.close()
html

'<!DOCTYPE html>\n<html class="desktop withSiteHeaderTopFullImage\n">\n<head>\n  <title>Javier’s books on Goodreads (575 books)</title>\n\n<meta content=\'Javier has 575 books on his all shelf: How to Start a Successful Airbnb Business: Quit Your Day Job and Earn Full-time Income on Autopilot With a Profita...\' name=\'description\'>\n<meta content=\'telephone=no\' name=\'format-detection\'>\n<link href=\'https://www.goodreads.com/review/list/50196372\' rel=\'canonical\'>\n  <meta property="og:title" content="Javier’s books on Goodreads (575 books)"/>\n  <meta property="og:type" content="website"/>\n  <meta property="og:site_name" content="Goodreads"/>\n  <meta property="og:description" content="Javier has 575 books on his all shelf: How to Start a Successful Airbnb Business: Quit Your Day Job and Earn Full-time Income on Autopilot With a Profita..."/>\n    <meta property="og:image" content="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1634021238l/59335141._SY4

In [ ]:
# pip3 install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup

# the URL of the login page
login_url="".join([
  "https://www.goodreads.com/ap/signin"
	,"?openid.assoc_handle=amzn_goodreads_web_na"
	,"&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select"
	,"&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select"
	,"&openid.mode=checkid_setup"
	,"&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0"
	,"&openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fsign-in"
])
print(url)


curls = """
curl 'https://www.goodreads.com/ap/signin' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/x-www-form-urlencoded' \
  -H 'Origin: https://www.goodreads.com' \
  -H 'Referer: https://www.goodreads.com/ap/signin?openid.assoc_handle=amzn_goodreads_web_na&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fsign-in&openid.mode=checkid_setup' \
  --data-raw 'email=dark_javier_haf%40hotmail.com&create=0&encryptedPwd=AYAAFK1SrwhdkoHgyjQkiRPgRtUAAAABAAZzaTptZDUAIDU2ZDE0ZWRjZThlMmNiNmM2ODQyYzU5ZGRhZWU0MjZlAQA8R05WFgPEHhg26BDMzgC5MK86aKYGWiTLJ87EqKG8gLd7dsduK7qcLSBuN0p7DRz6V5B6kB2nv0gVz%2BWYC33DCpYFtVqhYb%2FfPdUZv6hecxBNvOVlCebAn4yY1YLIZEs9DSppG6JJ8e8I%2FGknn0iWZxb7JvB2nV6vTvDRmCrtkoHtflGanF5qIXgH6sdXhHXOMMLRC8sINis6i8ocHcH4iOCZPWIyIR7cx%2BUatOIoWm2lY0Q9WxtGoFncYt9dBLRGtUcwtEeIYywdgTx7JuwUODdhSCU4coni9QDE80l1KW8I3H4a8WL8h9z7hbxBqQTSMCN2m2gE4wm07aI6IYsYAgAAAAAMAAAAFQAAAAAAAAAAAAAAAFZ84X8aSr4UtVzPDDzXDOD%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAABRclAHV%2FtUOo0zxkfTo%2FcoZwNhTxfrqlPveJeu6JruCrINoJv4%3D'

curl 'https://www.goodreads.com/review/update_session_shelf_settings' \
  -H 'Accept: text/javascript, text/html, application/xml, text/xml, */*' \
  -H 'Connection: keep-alive' \
  -H 'Content-type: application/x-www-form-urlencoded; charset=UTF-8' \
  -H 'Origin: https://www.goodreads.com' \
  -H 'Referer: https://www.goodreads.com/review/list/50196372-javier' \
  --data-raw 'utf8=%E2%9C%93&shelf%5Bdisplay_fields%5D%5Bauthor%5D=1&shelf%5Bdisplay_fields%5D%5Bavg_rating%5D=1&shelf%5Bdisplay_fields%5D%5Bcover%5D=1&shelf%5Bdisplay_fields%5D%5Bdate_added%5D=1&shelf%5Bdisplay_fields%5D%5Bdate_pub%5D=1&shelf%5Bdisplay_fields%5D%5Bnum_pages%5D=1&shelf%5Bdisplay_fields%5D%5Bnum_ratings%5D=1&shelf%5Bdisplay_fields%5D%5Brating%5D=1&shelf%5Bdisplay_fields%5D%5Bshelves%5D=1&shelf%5Bdisplay_fields%5D%5Btitle%5D=1&shelf%5Bdisplay_fields%5D%5Bactions%5D=1'
"""
# the payload with your login credentials
payload = {
    "email": "admin@example.com",
    "password": "password",
}

# send the POST request to login
response = requests.post(login_url, data=payload)


# if the request went Ok, you should get a 200 status
print(f"Status code: {response.status_code}")

# parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# find the page title
page_title = soup.title.string

# print the result page title
print(f"Page title: {page_title}")